In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
import csv

from gpu import set_gpu

import tensorflow as tf
from latent.ml.dl_subclass import MLP, MLPbeta, CNN, MLPprop, get_train
from latent.ml.dl_manual import nn_pass, conv, bn
import latent.session_subclass as session
import latent.utils.data_utils as prd
from latent.ml.lda import train_lda, eval_lda

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
data, params = prd.load_caps_train('traindata.mat')

In [3]:
ep = 30
n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
mlp_prop = MLPprop(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_prop_accuracy = tf.keras.metrics.MeanSquaredError(name='train_prop_accuracy')

# Train neural networks
models = [mlp, mlp_beta, cnn]
models = [cnn]

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(data, params)

for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
 
    prop_b = isinstance(model, MLPprop)
 
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f}, '
                f'MSE: {train_prop_accuracy.result():.2f} '
            )
            
    del train_mod

# Train aligned LDA
# y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

# mlp_enc = mlp.get_layer(name='enc')
# w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

# mlpbeta_enc = mlp_beta.get_layer(name='enc')
# w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

# cnn_enc = cnn.get_layer(name='enc')
# temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
# temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
# w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# # Train LDA
# w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
# w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn.get_weights()
mlpp_w = mlp_prop.get_weights()

Epoch 1, Loss: 1.46, Accuracy: 52.35, MSE: 0.00 
Epoch 30, Loss: 0.05, Accuracy: 98.46, MSE: 0.00 


In [4]:
enc = cnn.get_layer(name='enc')

In [25]:
x = x_train_cnn[:2,...]
test = enc.conv1(x)
test2 = enc.bn1(test)
test3 = enc.conv2(test2)
flat = enc.flatten(test3)
conv_w = enc.conv1.get_weights()
conv2_w = enc.conv2.get_weights()
bn_w = enc.bn1.get_weights()
bn2_w = enc.bn2.get_weights()
den_w = enc.dense1.get_weights()
bn3_w = enc.bn3.get_weights()

In [26]:
out = conv(x,conv_w)
out1 = bn(out,bn_w)
out2 = conv(out1,conv2_w)
flat2 = np.reshape(out2,(x.shape[0],-1))

In [28]:
arch = ['CONV1','BN1','CONV2','BN2','FLAT','DEN','BN4','LIN','SOFTMAX']
arch = ['conv','bn','conv','bn','flat','den','bn','lin','softmax']
out = nn_pass(x,cnn_w,arch)



(2, 816)
(816, 16)
(2, 16)
(16, 4)
(2, 4)
(4,)


ValueError: operands could not be broadcast together with shapes (2,) (4,) 

In [6]:
# Save neural network architectures into csv
arch = ['RELU1','BN1','RELU2','BN2','RELU3','BN3','DEN','BN4','SOFTMAX','PROP']
w = {}
w_in = mlpp_w
folder = 'weights_p'
path = 'python/'

i = 0
for l in arch:
    w_layer = []
    if 'BN' in l:
        w_layer = np.vstack((np.vstack((np.vstack((w_in[i],w_in[i+1])),w_in[i+2])),w_in[i+3]))
        i += 4
    else:
        w_layer = np.vstack((w_in[i],w_in[i+1]))
        i += 2
    
    # Save to current folder
    with open(path + folder + '/' + l + '.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(w_layer)

    # Save to embedded
    try:
        with open('//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/' + folder + '/'  + l + '.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            # write multiple rows
            writer.writerows(w_layer)
    except:
        print('no embedded')
    w[l] = w_layer

fill = len(emg_scale) - len(x_min)
temp = np.hstack((np.vstack((x_min[...,np.newaxis],np.zeros((fill,1)))), np.vstack((x_max[...,np.newaxis],np.zeros((fill,1))))))
scales = np.hstack((emg_scale,temp))

# Save to current folder
with open(path + folder + '/ARCH.csv', 'w', newline='') as f:
        writer = csv.writer(f,delimiter=',')
        # write multiple rows
        for i in arch:
            writer.writerow([i])

with open(path + folder + '/scales.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(scales)

# Save to embedded
try:
    with open('//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/' + folder + '/ARCH.csv', 'w', newline='') as f:
            writer = csv.writer(f,delimiter=',')
            # write multiple rows
            for i in arch:
                writer.writerow([i])

    with open('//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/' + folder + '/scales.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            # write multiple rows
            writer.writerows(scales)
except:
    print('no embedded')

no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
